--- start connection ----

In [3]:
import requests
import pandas as pd
import sqlite3
import json
import sqlalchemy

In [4]:
#Code that gives us the first 10 rows from the API
payload = {'$limit':25}
r = requests.get('https://data.cityofnewyork.us/resource/erm2-nwe9.json',params=payload)

In [5]:
r.json()

[{'unique_key': '53780140',
  'created_date': '2022-03-30T02:07:15.000',
  'agency': 'NYPD',
  'agency_name': 'New York City Police Department',
  'complaint_type': 'Blocked Driveway',
  'descriptor': 'No Access',
  'location_type': 'Street/Sidewalk',
  'incident_zip': '11214',
  'incident_address': '1969 85 STREET',
  'street_name': '85 STREET',
  'cross_street_1': '19 AVENUE',
  'cross_street_2': '20 AVENUE',
  'intersection_street_1': '19 AVENUE',
  'intersection_street_2': '20 AVENUE',
  'address_type': 'ADDRESS',
  'city': 'BROOKLYN',
  'landmark': '85 STREET',
  'status': 'In Progress',
  'community_board': '11 BROOKLYN',
  'bbl': '3063330049',
  'borough': 'BROOKLYN',
  'x_coordinate_state_plane': '984687',
  'y_coordinate_state_plane': '159949',
  'open_data_channel_type': 'MOBILE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'BROOKLYN',
  'latitude': '40.60569937845837',
  'longitude': '-73.99842620656881',
  'location': {'latitude': '40.60569937845837',
   'longi

In [6]:
#Create our Database if it doesn't exist, and connect to it
con = sqlite3.connect('calls.db')

In [7]:
#Create a cursor to execute SQL to our Database
cur = con.cursor()

In [6]:
sql_create_raw_json_table = '''CREATE TABLE raw_json(
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                raw_text text
                            )'''

In [8]:
cur.execute(sql_create_raw_json_table)

NameError: name 'sql_create_raw_json_table' is not defined

In [8]:
con.commit()

In [9]:
[(json.dumps(x),) for x in r.json()]

[('{"unique_key": "53780140", "created_date": "2022-03-30T02:07:15.000", "agency": "NYPD", "agency_name": "New York City Police Department", "complaint_type": "Blocked Driveway", "descriptor": "No Access", "location_type": "Street/Sidewalk", "incident_zip": "11214", "incident_address": "1969 85 STREET", "street_name": "85 STREET", "cross_street_1": "19 AVENUE", "cross_street_2": "20 AVENUE", "intersection_street_1": "19 AVENUE", "intersection_street_2": "20 AVENUE", "address_type": "ADDRESS", "city": "BROOKLYN", "landmark": "85 STREET", "status": "In Progress", "community_board": "11 BROOKLYN", "bbl": "3063330049", "borough": "BROOKLYN", "x_coordinate_state_plane": "984687", "y_coordinate_state_plane": "159949", "open_data_channel_type": "MOBILE", "park_facility_name": "Unspecified", "park_borough": "BROOKLYN", "latitude": "40.60569937845837", "longitude": "-73.99842620656881", "location": {"latitude": "40.60569937845837", "longitude": "-73.99842620656881", "human_address": "{\\"addres

In [10]:
#json.loads(some_json)
#json.dumps(some_json)


list_of_strings = []
for row in r.json():
    list_of_strings.append(json.dumps(row))
list_of_strings

list_of_tuples = []
for row in list_of_strings:
    list_of_tuples.append((row,))
list_of_tuples

[('{"unique_key": "53780140", "created_date": "2022-03-30T02:07:15.000", "agency": "NYPD", "agency_name": "New York City Police Department", "complaint_type": "Blocked Driveway", "descriptor": "No Access", "location_type": "Street/Sidewalk", "incident_zip": "11214", "incident_address": "1969 85 STREET", "street_name": "85 STREET", "cross_street_1": "19 AVENUE", "cross_street_2": "20 AVENUE", "intersection_street_1": "19 AVENUE", "intersection_street_2": "20 AVENUE", "address_type": "ADDRESS", "city": "BROOKLYN", "landmark": "85 STREET", "status": "In Progress", "community_board": "11 BROOKLYN", "bbl": "3063330049", "borough": "BROOKLYN", "x_coordinate_state_plane": "984687", "y_coordinate_state_plane": "159949", "open_data_channel_type": "MOBILE", "park_facility_name": "Unspecified", "park_borough": "BROOKLYN", "latitude": "40.60569937845837", "longitude": "-73.99842620656881", "location": {"latitude": "40.60569937845837", "longitude": "-73.99842620656881", "human_address": "{\\"addres

In [11]:
cur.executemany("INSERT INTO raw_json (raw_text) VALUES (?)",list_of_tuples)

In [12]:
con.commit()

In [13]:
con.close()

In [14]:
con = sqlite3.connect('calls.db')
cur = con.cursor()

In [15]:
cur.execute("select * from raw_json")


In [16]:
x = cur.fetchall()

In [17]:
for row in x:
    print(row)

(1, '{"unique_key": "53740100", "created_date": "2022-03-26T02:06:14.000", "agency": "NYPD", "agency_name": "New York City Police Department", "complaint_type": "Noise - Commercial", "descriptor": "Loud Music/Party", "location_type": "Club/Bar/Restaurant", "incident_zip": "11216", "incident_address": "307 CLIFTON PLACE", "street_name": "CLIFTON PLACE", "cross_street_1": "NOSTRAND AVENUE", "cross_street_2": "MARCY AVENUE", "intersection_street_1": "NOSTRAND AVENUE", "intersection_street_2": "MARCY AVENUE", "address_type": "ADDRESS", "city": "BROOKLYN", "landmark": "CLIFTON PLACE", "status": "In Progress", "community_board": "03 BROOKLYN", "bbl": "3017890001", "borough": "BROOKLYN", "x_coordinate_state_plane": "997932", "y_coordinate_state_plane": "190380", "open_data_channel_type": "ONLINE", "park_facility_name": "Unspecified", "park_borough": "BROOKLYN", "latitude": "40.68921531863523", "longitude": "-73.95066450185375", "location": {"latitude": "40.68921531863523", "longitude": "-73.9

In [18]:
con.close()

In [19]:
#This function is going to get us our raw data

def get_raw_data():
    con = sqlite3.connect('calls.db')
    cur = con.cursor()
    list_of_rows = []
    for row in cur.execute('SELECT * from raw_json'):
        list_of_rows.append(row)
    con.close()
    return list_of_rows

In [20]:
raw_data = get_raw_data()

In [21]:
raw_data

[(1,
  '{"unique_key": "53740100", "created_date": "2022-03-26T02:06:14.000", "agency": "NYPD", "agency_name": "New York City Police Department", "complaint_type": "Noise - Commercial", "descriptor": "Loud Music/Party", "location_type": "Club/Bar/Restaurant", "incident_zip": "11216", "incident_address": "307 CLIFTON PLACE", "street_name": "CLIFTON PLACE", "cross_street_1": "NOSTRAND AVENUE", "cross_street_2": "MARCY AVENUE", "intersection_street_1": "NOSTRAND AVENUE", "intersection_street_2": "MARCY AVENUE", "address_type": "ADDRESS", "city": "BROOKLYN", "landmark": "CLIFTON PLACE", "status": "In Progress", "community_board": "03 BROOKLYN", "bbl": "3017890001", "borough": "BROOKLYN", "x_coordinate_state_plane": "997932", "y_coordinate_state_plane": "190380", "open_data_channel_type": "ONLINE", "park_facility_name": "Unspecified", "park_borough": "BROOKLYN", "latitude": "40.68921531863523", "longitude": "-73.95066450185375", "location": {"latitude": "40.68921531863523", "longitude": "-7

In [22]:
kvs_i_care_about = ['created_date','agency','agency_name','complaint_type','descriptor',
                   'location_type','incident_zip','incident_address','borough']

In [23]:
# takes in list of tuples (from a select statement), and
# it's going to prune the features we don't need.
def data_extractor(list_of_tuples):
    
    #Create an empty list to store the unpacked dictionaries.
    list_of_relevant_info = []
    
    
    #Contains the keys I care about. 
    #In future iterations, make this a parameter of the function.
    kvs_i_care_about = ['created_date','agency','agency_name','complaint_type','descriptor',
                   'location_type','incident_zip','incident_address','borough']
    
    
    
    #Driver code to unpack the tuples. 
    #The ID is going to be stored into X, and the Stringified JSON is going to 
    #be stored in Y    
    for x,y in list_of_tuples:
        
        
        #Create a dictionary that is going to hold the pruned K,V pairs.
        temp_dict = {}
        
        
        #Unstringifying the JSON, and storing it into a temporary variable
        parsed_json = json.loads(y)
        
        
        #Unpacking the JSON, storing keys into K, and the Values 
        #Into V
        for k,v in parsed_json.items():
            
            #If it finds the same key in my other list kvs_i_care about
            #It's going to create the K,V pairing in the temp_dict
            if k in kvs_i_care_about:
                temp_dict[k] = v
                
            #after it has checked all the K,V pairs, append it to the list
            #and go on to the next row
        list_of_relevant_info.append(temp_dict)
            
    return list_of_relevant_info

In [24]:
cleaned_data = data_extractor(get_raw_data())

In [25]:
cleaned_data

[{'created_date': '2022-03-26T02:06:14.000',
  'agency': 'NYPD',
  'agency_name': 'New York City Police Department',
  'complaint_type': 'Noise - Commercial',
  'descriptor': 'Loud Music/Party',
  'location_type': 'Club/Bar/Restaurant',
  'incident_zip': '11216',
  'incident_address': '307 CLIFTON PLACE',
  'borough': 'BROOKLYN'},
 {'created_date': '2022-03-26T02:06:05.000',
  'agency': 'NYPD',
  'agency_name': 'New York City Police Department',
  'complaint_type': 'Blocked Driveway',
  'descriptor': 'No Access',
  'location_type': 'Street/Sidewalk',
  'incident_zip': '10458',
  'incident_address': '464 EAST  185 STREET',
  'borough': 'BRONX'},
 {'created_date': '2022-03-26T02:05:39.000',
  'agency': 'NYPD',
  'agency_name': 'New York City Police Department',
  'complaint_type': 'Noise - Residential',
  'descriptor': 'Loud Music/Party',
  'location_type': 'Residential Building/House',
  'incident_zip': '10458',
  'incident_address': '2881 VALENTINE AVENUE',
  'borough': 'BRONX'},
 {'cr

In [26]:
sql_create_cleaned_table = '''CREATE TABLE calls(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    created_data text,
                    agency text,
                    agency_name text,
                    complaint_type text,
                    descriptor text,
                    location_type text,
                    incident_zip text,
                    incident_address text,
                    borough text
)
'''

In [27]:
con = sqlite3.connect('calls.db')
cur = con.cursor()
cur.execute(sql_create_cleaned_table)
con.close()

In [28]:
# take in a list of dictionaries (result of data_extractor)
# It's going to give a list of tuples, where the tuples are the values from the dictionaries

def json_extractor(list_of_dicts):
    # initialize a list that's going to hold all of my tuple pairs
    list_of_tuples = []

    
    #iterate over the list of dictionaries
    for row in list_of_dicts:    
        
        
        #Create a tuple of all the values within that row
        temp_list = tuple(x for x in row.values())
        
        #Append the tuple to the list
        list_of_tuples.append(temp_list)
        
    return list_of_tuples

In [29]:
holder = json_extractor(cleaned_data)
holder

[('2022-03-26T02:06:14.000',
  'NYPD',
  'New York City Police Department',
  'Noise - Commercial',
  'Loud Music/Party',
  'Club/Bar/Restaurant',
  '11216',
  '307 CLIFTON PLACE',
  'BROOKLYN'),
 ('2022-03-26T02:06:05.000',
  'NYPD',
  'New York City Police Department',
  'Blocked Driveway',
  'No Access',
  'Street/Sidewalk',
  '10458',
  '464 EAST  185 STREET',
  'BRONX'),
 ('2022-03-26T02:05:39.000',
  'NYPD',
  'New York City Police Department',
  'Noise - Residential',
  'Loud Music/Party',
  'Residential Building/House',
  '10458',
  '2881 VALENTINE AVENUE',
  'BRONX'),
 ('2022-03-26T02:05:26.000',
  'NYPD',
  'New York City Police Department',
  'Noise - Residential',
  'Loud Music/Party',
  'Residential Building/House',
  '11207',
  '224 HIGHLAND BOULEVARD',
  'BROOKLYN'),
 ('2022-03-26T02:05:09.000',
  'NYPD',
  'New York City Police Department',
  'Noise - Residential',
  'Loud Music/Party',
  'Residential Building/House',
  '10458',
  '2881 VALENTINE AVENUE',
  'BRONX'),
 (

In [30]:
con = sqlite3.connect('calls.db')
cur = con.cursor()
cur.executemany("INSERT INTO calls (created_data,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,borough) VALUES (?,?,?,?,?,?,?,?,?)",holder)
con.commit()
con.close()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 9, and there are 7 supplied.

In [ ]:
def return_rows_by_borough(borough_name):
    con = sqlite3.connect('calls.db')
    cur = con.cursor()
    dict_to_pass = {'x':borough_name}
    list_of_rows = [x for x in cur.execute('select * from calls where borough=:x',dict_to_pass)]
    con.close()
    if list_of_rows:
        return list_of_rows
    else:
        return 'No Matches Found'

In [ ]:
return_rows_by_borough('DROP TABLES calls')

'No Matches Found'

-- end connection --- 